In [1]:
from scripts.StreamProcessing import Stream_Data # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

PySpark found


https://berkeleyearth.org/data/

### A note on block 

These might take a while as PySpark attempts to download some additional files

In [2]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
emissions_config = Stream_Data(topics='ghg_data', host='localhost:9092')
e_session, query, emissions_df = emissions_config.getData()
#temperatures

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/mysql-connector-j-8.0.33.jar
/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/graphframes-0.8.2-spark3.2-s_2.12.jar


23/05/15 23:48:14 WARN Utils: Your hostname, camagakhan-Inspiron-7577 resolves to a loopback address: 127.0.1.1; using 192.168.6.69 instead (on interface wlp60s0)
23/05/15 23:48:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camagakhan/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camagakhan/.ivy2/cache
The jars for the packages stored in: /home/camagakhan/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aeeac8ca-c670-43f2-9738-70f1a9117f29;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#com

In [3]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
temperature_config = Stream_Data(topics='temperature', host='localhost:9092')
t_session, query_temperature, temperature_df = temperature_config.getData()

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/mysql-connector-j-8.0.33.jar
/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/graphframes-0.8.2-spark3.2-s_2.12.jar


### NOTE
Everything is configered while initializing the PySpark Session in the ```StreamProcessing.py``` class. DO NOT remove the ```from graphframes import GraphFrame```, as the jar files are loaded while getting the data from the previous code block

In [18]:
from graphframes import GraphFrame
import findspark as tt
tt.init(emissions_config.getSpark_Path()) 
from pyspark.sql import functions as F

In [5]:
print(temperature_df)
print(emissions_df)

DataFrame[REF_AREA: string, Measure: string, UNIT_MEASURE: string, TIME_PERIOD: int, OBS_VALUE: double, REF_CODE: string]
DataFrame[PREFIX: string, Country: string, POL: string, Pollutant: string, VAR: string, Variable: string, YEA: int, Year: int, UnitCode: string, Unit: string, PowerCodeCode: string, PowerCode: string, ReferencePeriodCode: float, ReferencePeriod: float, Value: double, FlagCodes: string, Flags: string]


In [22]:
temp_df = temperature_df.withColumnRenamed('REF_CODE','COUNTRY_CODE')
ghg_df = emissions_df.withColumnRenamed('PREFIX', 'COUNTRY_CODE')
joined_dataset = temp_df.join(ghg_df, (temp_df.COUNTRY_CODE == ghg_df.COUNTRY_CODE) & (temp_df.TIME_PERIOD == ghg_df.Year) & (ghg_df.COUNTRY_CODE == 'MLT'), 'inner')


joined_dataset = joined_dataset.select(temp_df.COUNTRY_CODE, ghg_df.Country,ghg_df.Year, temp_df.UNIT_MEASURE, temp_df.OBS_VALUE, ghg_df.POL, ghg_df.VAR, ghg_df.Value)

(joined_dataset.writeStream 
   .format('console').start())



# ran into out of memory exception

# We'll be assigning the country_codes as ids to map relationships

-------------------------------------------
Batch: 0
-------------------------------------------
+------------+-------+----+------------------+---------+---+------------+--------+
|COUNTRY_CODE|Country|Year|      UNIT_MEASURE|OBS_VALUE|POL|         VAR|   Value|
+------------+-------+----+------------------+---------+---+------------+--------+
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|      ENER_P|  92.454|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|      LULUCF|  -8.152|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|  ENER_OSECT| 265.186|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|    ENER_IND|1765.507|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|        ENER|2403.135|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|  INDEX_2000|  93.174|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|ENER_TRANS_P|  12.251|
|         MLT|  Malta|1990|C: Degrees celsius|     0.44|GHG|         AGR|